# Initialization

In [2]:
import pandas as pd
from tqdm import tqdm
from enum import Enum
from statistics_helper import Helper, read_pfm
import json
import os
import numpy as np
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from const import Const
from position_helper import rank_persons


face_log_path = ''
depth_log_path = ''
final_dir = 'results'

if face_log_path == '':
    raise ValueError("please set face_log_path")
if depth_log_path == '':
    raise ValueError("please set depth_log_path")


imdir_ind = face_log_path.rfind('_')
dot_ind = face_log_path.find('.')
date = face_log_path[imdir_ind + 1: dot_ind]

img_df = pd.read_csv(face_log_path)
depth_df = pd.read_csv(depth_log_path)
helper = Helper()

num_bad_images = 0
total_faces = 0
total_mixed_images = 0
dominant_gender_ind = 0
dominant_race_ind = 1
dominant_emotion_ind = 2
dominant_age_range_ind = 3
img_dim = {'width': 1024, 'height': 1024}
image_center_x = img_dim['width'] / 2 
image_center_y = img_dim['height'] / 2


ValueError: please set face_log_path

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)  # Convert np.float32 to Python's native float

        return json.JSONEncoder.default(self, obj)

In [ ]:
pbar = tqdm(img_df.itertuples(), total=len(img_df), position=0, leave=True, desc="images")
for row in pbar:
    try:
        if type(row.demography) == float and math.isnan(row.demography):
            num_bad_images = num_bad_images + 1
            continue
        demography = json.loads(row.demography)
        num_faces = len(demography)
        if num_faces == 1:
            continue
        
        filtred_pfm = depth_df[(depth_df['image_id'] == row.image_id) & (depth_df['pfm'] == True) & (depth_df['with_negative_prompt'] == row.with_negative_prompt)]
        
        pfm = read_pfm(filtred_pfm.iloc[0]['path'])[0]
        max_depth = int(np.max(pfm))
        min_depth = int(np.min(pfm))
        total_faces = total_faces + num_faces
        face_info = []
        face_combined_info = []
        # helper.add_number_faces(num_faces)
        for face in demography:
            x = face['region']['x']
            y = face['region']['y']
            w = face['region']['w']
            h = face['region']['h']
            depth_roi = pfm[y : y + h, x : x + w]
            face_center_x = x + (w / 2)
            face_center_y = y + (h / 2)
            delta_x = face_center_x - image_center_x
            delta_y = face_center_y - image_center_y
            centrality = math.sqrt(delta_x**2 + delta_y**2)
            mean_depth_img = int(np.mean(depth_roi))
            max_depth_img = int(np.max(depth_roi))
            dominant_gender = face[Const.dominant_gender]
            dominant_race = face[Const.dominant_race]
            dominant_emotion = face[Const.dominant_emotion]
            age = face[Const.age]
            # fill race
            helper.add_gender_age( dominant_gender, age)
            helper.add_gender_emotion( dominant_gender, dominant_emotion)
            helper.add_gender_race( dominant_gender, dominant_race)
            helper.add_gender_total_num_faces( dominant_gender)
            helper.add_race_age( dominant_race, age)
            helper.add_race_emotion( dominant_race, dominant_emotion)
            helper.add_race_gender( dominant_race, dominant_gender)
            helper.add_race_total_num_faces( dominant_race)
            age_range = int(age / 10)
            face_combined_info.append({Const.num_faces: num_faces,
                                        Const.combined: {
                                            Const.age: age,
                                            Const.gender: dominant_gender,
                                            Const.race: dominant_race,
                                            Const.emotion: dominant_emotion,
                                            Const.face_position_per: mean_depth_img,
                                            Const.face_position_norm: -1,
                                            Const.age_range: f'{age_range * 10} - {age_range * 10 + 9} years',
                                            Const.x: x,
                                            Const.y: y,
                                            Const.w: w,
                                            Const.h: h,
                                            Const.face_center_x: face_center_x,
                                            Const.face_center_y: face_center_y,
                                            Const.centrality: centrality,
                                            Const.mean_depth: mean_depth_img,
                                            Const.mixed_gender: False,
                                            Const.mixed_race: False,
                                            Const.mixed_emotion: False,
                                            Const.mixed_ages: False}})
            face_info.append((dominant_gender, dominant_race, dominant_emotion, f'{age_range * 10} - {age_range * 10 + 9} years'))
        
        if num_faces == 1:
            continue
        
        face_combined_info.sort(key=lambda x: x[Const.combined][Const.face_position_per], reverse=True)
        face_combined_info = rank_persons(face_combined_info, 0.07)

        dominant_genders = {t[dominant_gender_ind] for t in face_info}
        dominant_races = {t[dominant_race_ind] for t in face_info}
        dominant_emotions = {t[dominant_emotion_ind] for t in face_info}
        dominant_age_ranges = {t[dominant_age_range_ind] for t in face_info}
        
        for gender in Const.possible_values[Const.gender]:
            if gender in dominant_genders:
                helper.add_gender_total_apparences(gender)

        for race in Const.possible_values[Const.race]:
            if race in dominant_races:
                helper.add_race_total_apparences(race)
        
        
        
        if len(dominant_genders) > 1 or len(dominant_races) > 1:
            total_mixed_images += 1
            
        if len(dominant_emotions) > 1:
            for _dict in face_combined_info:
                _dict[Const.combined][Const.mixed_emotion] = True
                
        if len(dominant_age_ranges) > 1:
            for _dict in face_combined_info:
                _dict[Const.combined][Const.mixed_ages] = True

        if len(dominant_genders) > 1:

            for _dict in face_combined_info:
                _dict[Const.combined][Const.mixed_gender] = True
                
            for gender in Const.possible_values[Const.gender]:
                if gender in dominant_genders:
                    helper.add_gender_total_apparences_mixed(gender)

        if len(dominant_races) > 1:
            for _dict in face_combined_info:
                _dict[Const.combined][Const.mixed_race] = True
            
            for race in Const.possible_values[Const.race]:
                if race in dominant_races:
                    helper.add_race_total_apparences_mixed(race)
                
        for _dict in face_combined_info:
            helper.add_combined_info(_dict[Const.combined])    
        
    except ValueError:
        num_bad_images = num_bad_images + 1
        print(f'already found {num_bad_images} bad images of {index + 1} in total.')
    
helper.set_bad_imgs(num_bad_images)
helper.set_total_faces(total_faces)
helper.set_total_imgs(len(img_df))
helper.set_mixed_imgs(mixed_imgs=total_mixed_images)

helper.to_json(os.path.join(final_dir, f'demographies_{date}.json'))